In [1]:
type ∨[P, Q] = Either[P, Q]
type ∧[P, Q] = Tuple2[P, Q]
type ⟶[P, Q] = P => Q
type ⊥ = Nothing
type True = Unit
type ¬[P] = P => ⊥
type ⟷[P, Q] = (P => Q) ∧ (Q => P)

defined type ∨
defined type ∧
defined type ⟶
defined type ⊥
defined type True
defined type ¬
defined type ⟷

In [2]:
type Or[P, Q] = Either[P, Q]
type And[P, Q] = Tuple2[P, Q]
type Implies[P, Q] = P => Q
type False = Nothing
type True = Unit
type Not[P] = P => ⊥
type <=>[P, Q] = (P => Q, Q => P)

defined type Or
defined type And
defined type Implies
defined type False
defined type True
defined type Not
defined type <=>

In [3]:
class Inhabitant{ x => 
    // Knight(x) -- `x` is a Knight
    // 
    type Knight
    
    // Knave(x) -- `x` is a Knave (i.e. is not a Knight)
    // 
    type Knave = ¬[Knight]
    
    // Says(x, P) -- `x` says that `P` holds, i.e. asserts proposition `P`
    // 
    type Says[P]
    
    // Tipo Arthur York
    //
    type Arthur
    
    // Being always truthful
    type AlwaysTruthful = Knight    
}

defined class Inhabitant

In [4]:
trait KnightsKnaves{
    // P1. Inhabitants are knights or knaves
    // 
    //     ∀ x. Inhabitant(x) ⟶ Knight(x) ∨ Knave(x) 
    // 
    def P1(x: Inhabitant): x.Knight ∨ x.Knave
    
    // In Scala 3
    // val P1: (x: Inhabitant) => Either[x.Knight, x.Knave]
    
    // P2. Knights are truth tellers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ P
    // 
    def P2[P](x: Inhabitant): x.Knight => x.Says[P] => P
    
    // In Scala 3
    // val P2: [P] => (x: Inhabitant) => x.Knight => x.Says[P] => P
    
    // P3. Knaves are persistent liers
    // 
    //     ∀ P. ∀ x. Knight(x) ⟶ Says(x, P) ⟶ ¬P
    // 
    def P3[P](x: Inhabitant): x.Knave => x.Says[P] => ¬[P]
    
    /*P4. Si x dice que es Arthur York, sera un caballero porque dice la verdad - ¿?¿?
    def P4[P](x: Inhabitant): x.Arthur => x.Knight
    
    Tip de Juanma
    Si el sujeto es Arthur, entonces si Arthur dice P, quiere decir que el sujeto dice P
    def P4_1[P](x: Inhabitant)(arthur: Inhabitant): x.Arthur => arthur.Says[P] => x.Says[P]
    
    Si el sujeto es Arthur, entonces si Arthur es un Knight, quiere decir que el sujeto es un Knight
    def P4_2[P](x: Inhabitant)(arthur: Inhabitant): x.Arthur => arthur.Knight => x.Knight
    
    */
    
    //P4. If some inhabitant is actually Arthur York, all that can be proven about Arthur York can be proven about her
    val arthur: Inhabitant
    
    def P4_1[P](x: Inhabitant): x.Arthur => arthur.Says[P] => x.Says[P]
    def P4_2(x: Inhabitant): x.Arthur => arthur.Knight => x.Knight
    
    // P5. Double negation applies to things said
    def P5[P](x: Inhabitant): Not[Not[x.Says[P]]] => x.Says[P]
    
    // P6. Some inhabitant is or is not Arthur York
    def P6(x: Inhabitant): x.Arthur Or Not[x.Arthur]  
    
    // P7. Si x dice la verdad, x es Knight
    def P7[P](x: Inhabitant): x.Says[P] => P => x.Knight
    
    // P8. Si x miente, x es Knave
    def P8[P](x: Inhabitant): x.Says[P] => Not[P] => x.Knave
    
}

defined trait KnightsKnaves

In [25]:
implicit class SomeSugar(P: KnightsKnaves){    
    def eitherKnightOrKnave(x: Inhabitant): x.Knight Or x.Knave = 
        P.P1(x)
    
    def knaveIfNotKnight(x: Inhabitant): Not[x.Knight] => x.Knave = 
        notXIsKnight => 
            eitherKnightOrKnave(x).fold(notXIsKnight, identity)
    
    def knightIfNotKnave(x: Inhabitant): Not[x.Knave] => x.Knight = 
        notXIsKnave => 
            eitherKnightOrKnave(x).fold(identity, notXIsKnave)
    
    def knightsAreTruthTellers[P](x: Inhabitant): x.Knight => x.Says[P] => P = 
        P.P2[P](x)
    
    def knavesAreLiers[P](x: Inhabitant): x.Knave => x.Says[P] => Not[P] = 
        P.P3[P](x)
    
    def doubleNotSayP[P](x: Inhabitant): Not[Not[x.Says[P]]] => x.Says[P] =
        P.P5[P](x)
    
    def eitherArthurOrNot(x: Inhabitant): x.Arthur Or Not[x.Arthur] = 
        P.P6(x)   
    
    def knightIfTruthful[P](x: Inhabitant): x.Says[P] => P => x.Knight = 
        xSaysP => p => 
            eitherKnightOrKnave(x).fold(
                (xIsKnight: x.Knight) => xIsKnight, 
                (xIsKnave: x.Knave) => knavesAreLiers(x)(xIsKnave)(xSaysP)(p)
            )
    
    def knaveIfLier[P](x: Inhabitant): x.Says[P] => Not[P] => x.Knave = 
        xSaysP => notP => eitherKnightOrKnave(x).fold(
            xIsKnight => notP(knightsAreTruthTellers(x)(xIsKnight)(xSaysP): P),
            identity)
}

implicit class NotSugar[P](notP: P => Nothing){
    def contradicts(p: P): Nothing = 
        notP(p)
}

defined class SomeSugar
defined class NotSugar

# Problem 2 - The Second Trial.
Another suspect was arrested and brought to trial. Here is a transcript of the trial:

Craig: The last suspect was a queer bird; he actually claimed to be Arthur York! Did you ever claim to be Arthur York?
DEFENDANT: NO.

Craig: Did you ever claim that you are not Arthur York?
DEFENDANT: Yes.

Craig’s first guess was that the defendant was not Arthur York, but are there really sufficient grounds for acquitting him?

**SOLUCION**

The defendant is either a knight or a knave. 

Suppose he is a knight. Then his answers were both truthful; in particular, his second answer was truthful, so he did once claim that he is not Arthur York. His claim was true, since he is a knight; thus he is not Arthur York. 
This proves that if he is a knight, then he is not Arthur York. 

Suppose he is a knave. Then his answers were both lies; in particular, his first answer was a lie, which means that he did once claim to be Arthur York. But since he is a knave, he lied when he claimed to be Arthur York, hence he is not Arthur York. And so we have proved that if he is a knave, then he is not Arthur York. 

We now see that regardless of **whether he is a knight or a knave, he cannot be Arthur York.** And so he was acquitted. Incidentally, **it cannot be determined whether he is a knight or a knave.**

In [25]:
/*
TERCER RAZONAMIENTO

(1)Craig: The last suspect was a queer bird; he actually claimed to be Arthur York! Did you ever claim to be Arthur York? 
DEFENDANT: NO.
(2)Craig: Did you ever claim that you are not Arthur York? 
DEFENDANT: Yes.
Craig’s first guess was that the defendant was not Arthur York, but are there really sufficient grounds for acquitting him?

De aqui saco que:

x = Defendant
arthur = Arthur

(1) x.Says[Not[x.Says[x.Arthur]]] -> El defendant dice que NO ha dicho que es Arthur York
(2) x.Says[x.Says[Not[x.Arthur]]] -> El defendant dice que ha dicho que NO es Arthur York

SOLUCION

The defendant is either a knight or a knave.
Suppose he is a knight: 
Then his answers were both truthful; in particular, his second answer was truthful, 
so he did once claim that he is not Arthur York. His claim was true, since he is a knight; thus he is not Arthur York. 
This proves that if he is a knight, then he is not Arthur York.

Suppose he is a knave: 
Then his answers were both lies; in particular, his first answer was a lie, 
which means that he did once claim to be Arthur York. But since he is a knave, 
he lied when he claimed to be Arthur York, hence he is not Arthur York. 
And so we have proved that if he is a knave, then he is not Arthur York.

We now see that regardless of whether he is a knight or a knave, 
he cannot be Arthur York. And so he was acquitted. 
Incidentally, it cannot be determined whether he is a knight or a knave.

Tanto si es un Knight, como un Knave, lo unico que se puede afirmar es que el defendant NO es Arthur York
(1) Si x.Knight -> Not[x.Arthur] 
(2) Si x.Knave -> Not[x.Arthur]

Por tanto:

PREDICADO                                                            SOLUCION
((x.Says[Not[x.Says[x.Arthur]]], x.Says[x.Says[Not[x.Arthur]]])) => Not[x.Arthur]

*/

In [7]:
def problem2(premises: KnightsKnaves)(x: Inhabitant): 
    ((x.Says[Not[x.Says[x.Arthur]]], x.Says[x.Says[Not[x.Arthur]]])) => Not[x.Arthur] = ???

defined function problem2

In [28]:
//FINALIZADO OK 
//Simplificado v3

def problem2(premises: KnightsKnaves)(x: Inhabitant): 
    ((x.Says[Not[x.Says[x.Arthur]]], x.Says[x.Says[Not[x.Arthur]]])) => Not[x.Arthur] = 
    xSay =>
        premises.eitherKnightOrKnave(x).fold(
            xIsKnight => {
                xIsArthur =>
                val xSayxIsNotArthur: x.Says[Not[x.Arthur]] = 
                    premises.knightsAreTruthTellers[x.Says[Not[x.Arthur]]](x)(xIsKnight)(xSay._2)
                val xIsNotArthur: Not[x.Arthur] = 
                    premises.knightsAreTruthTellers[Not[x.Arthur]](x)(xIsKnight)(xSayxIsNotArthur)
                xIsNotArthur(xIsArthur)
            },
            xIsKnave => {
                xIsArthur =>
                val xSayxIsArthur: x.Says[x.Arthur] = 
                    premises.doubleNotSayP[x.Arthur](x)(premises.knavesAreLiers[Not[x.Says[x.Arthur]]](x)(xIsKnave)(xSay._1))
                val xIsNotArthur: Not[x.Arthur] = 
                    premises.knavesAreLiers[x.Arthur](x)(xIsKnave)(xSayxIsArthur)
                xIsNotArthur(xIsArthur)                  
            }
        )

defined function problem2